# Working with Cultural Heritage APIs: Europeana

Welcome to this workshop on working with APIs and cultural heritage data. In this notebook, you will:

1. **Learn** what an API is and why APIs matter for digital humanities research
2. **Access** metadata from Europeana's aggregated collections (50+ million items)
3. **Explore** the structure of the metadata interactively
4. **Filter** artworks by creator, keyword, country, or other criteria
5. **Download** images of selected artworks
6. **Learn about** IIIF (International Image Interoperability Framework)

---

## About Europeana

[Europeana](https://www.europeana.eu/) is Europe's digital platform for cultural heritage, providing access to:
- **50+ million** digitized items from European museums, galleries, libraries and archives
- **Content from** 3,000+ institutions across Europe
- **Collections including** artworks, books, music, videos, photographs, manuscripts
- **Open data** under various Creative Commons licenses

Europeana aggregates content from major institutions including:
- Rijksmuseum (Netherlands)
- British Library (UK)
- Louvre (France)
- And many more across Europe

**Note:** Europeana uses IIIF (International Image Interoperability Framework) for many items, providing standardized access to high-resolution images.

---

## Part 1: What is an API?

**API** stands for **Application Programming Interface**

An API:
- Takes your **request** ("give me all paintings by Rembrandt")
- Sends it to a **server** (the database)
- Returns a **response** (the data you asked for)

### Why APIs matter for Digital Humanities

- **Scale**: Download thousands of records automatically instead of clicking through web pages
- **Structure**: Data comes in machine-readable formats (JSON, XML) ready for analysis
- **Reproducibility**: Your code documents exactly how you obtained your data
- **Updates**: Re-run your code to get the latest data
- **Integration**: Combine data from multiple institutions

### Common Data Formats

| Format | Description | Example |
|--------|-------------|---------|
| **JSON** | JavaScript Object Notation - human-readable, widely used | `{"name": "Mona Lisa", "year": 1503}` |
| **XML** | eXtensible Markup Language - similar to HTML | `<artwork><name>Mona Lisa</name></artwork>` |
| **CSV** | Comma-Separated Values - spreadsheet-like | `name,year\nMona Lisa,1503` |

---

## Part 2: Setup

First, let's import the libraries we need and set up our project structure.

In [ ]:
# Standard library imports
import os
import json
import time
from pathlib import Path
from urllib.parse import unquote

# External libraries (you may need to install these)
import requests
from IPython.display import display, Image, HTML

# Set up paths
PROJECT_ROOT = Path("../").resolve()
DATA_DIR = PROJECT_ROOT / "data" / "europeana"
IMAGES_DIR = PROJECT_ROOT / "images" / "europeana"

# Create directories if they don't exist
DATA_DIR.mkdir(parents=True, exist_ok=True)
IMAGES_DIR.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Data directory: {DATA_DIR}")
print(f"Images directory: {IMAGES_DIR}")

### API Key Configuration

Europeana requires a free API key. You can get one by:
1. Visit: https://pro.europeana.eu/page/get-api
2. Register for a Europeana account
3. Request an API key from your account section
4. Save your API key to `misc/api-key-europeana.txt`

For testing, we'll use a demo key with limited access.

In [ ]:
# Load API key if it exists
MISC_DIR = PROJECT_ROOT / "misc"
API_KEY_FILE = MISC_DIR / "api-key-europeana.txt"

# Default demo key (limited requests)
API_KEY = "api2demo"

if API_KEY_FILE.exists():
    with open(API_KEY_FILE, 'r') as f:
        custom_key = f.read().strip()
        if custom_key:
            API_KEY = custom_key
            print(f"✓ API key loaded from {API_KEY_FILE}")
else:
    print(f"ℹ Using demo API key (limited to 999 requests)")
    print(f"  For unlimited access, get your own key at: https://pro.europeana.eu/page/get-api")
    print(f"  Save it to: {API_KEY_FILE}")
    
# Create misc directory if it doesn't exist
MISC_DIR.mkdir(exist_ok=True)

# Base API endpoint
BASE_URL = "https://api.europeana.eu/record/v2"
print(f"\nAPI endpoint: {BASE_URL}")

---

## Part 3: Understanding the Europeana API

The Europeana API provides two main endpoints:

1. **Search API** - Query and filter the collection
2. **Record API** - Get detailed information about specific items

### Key Features
- **50+ million items** from European institutions
- **Filtering** by license, media type, country, institution
- **IIIF support** for many items (standardized image access)
- **Multilingual** metadata in various European languages

### Search API Parameters

| Parameter | Description | Example |
|-----------|-------------|---------|
| `query` | Search term | `Rembrandt`, `painting`, `*` (all) |
| `qf` | Query filter | `TYPE:IMAGE`, `COUNTRY:Netherlands` |
| `reusability` | License filter | `open`, `restricted`, `permission` |
| `rows` | Results per page (max 100) | `12` (default), `100` |
| `profile` | Detail level | `standard`, `rich` |

---

## Part 4: Searching the Europeana Collection

Let's create a function to search Europeana and explore the results.

In [ ]:
def search_europeana(query="*", rows=12, reusability="open", qf=None, profile="rich"):
    """
    Search the Europeana collection.
    
    Parameters:
        query: Search term (default: "*" for all)
        rows: Number of results to return (max 100)
        reusability: Filter by license ("open", "restricted", "permission", or None)
        qf: Additional query filters as list (e.g., ["TYPE:IMAGE", "COUNTRY:Netherlands"])
        profile: "standard" or "rich" for more metadata
    
    Returns:
        Dictionary with search results
    """
    url = f"{BASE_URL}/search.json"
    
    params = {
        "wskey": API_KEY,
        "query": query,
        "rows": min(rows, 100),  # Max 100
        "profile": profile
    }
    
    if reusability:
        params["reusability"] = reusability
    
    if qf:
        params["qf"] = qf
    
    try:
        response = requests.get(url, params=params, timeout=30)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"❌ Error searching Europeana: {e}")
        return None

# Test the search function
print("Testing Europeana API...")
test_results = search_europeana(query="Rembrandt", rows=3)

if test_results and test_results.get('success'):
    print(f"✓ API working! Found {test_results['totalResults']:,} items matching 'Rembrandt'")
    print(f"  Showing first {len(test_results['items'])} results")
else:
    print("❌ API test failed")

### Inspecting a Single Item

Let's look at one item to understand the data structure.

In [ ]:
# Get a sample item
if test_results and test_results['items']:
    sample_item = test_results['items'][0]
    print(json.dumps(sample_item, indent=2, ensure_ascii=False)[:2000], "...\n[truncated]")

### Understanding Europeana Metadata Fields

Europeana uses EDM (Europeana Data Model). Key fields:

| Field | Description | Example |
|-------|-------------|---------|
| `id` | Unique Europeana identifier | `/123/ABC456` |
| `title` | Item title (array) | `["The Night Watch"]` |
| `dcCreator` | Creator/artist name | `["Rembrandt van Rijn"]` |
| `year` | Year or date range | `["1642"]` |
| `edmPreview` | Thumbnail image URL | `https://...` |
| `edmIsShownBy` | Full-size image URL | `https://...` |
| `edmIsShownAt` | Link to source institution | `https://...` |
| `type` | Item type | `IMAGE`, `TEXT`, `VIDEO`, `SOUND`, `3D` |
| `rights` | License information | CC0, CC BY, etc. |
| `country` | Source country | `["Netherlands"]` |
| `dataProvider` | Source institution | `["Rijksmuseum"]` |

---

## Part 5: Interactive Data Exploration

Let's create helper functions to explore and filter the collection.

In [ ]:
def get_item_title(item):
    """Extract title from an item (handles multiple formats)."""
    if 'title' in item and item['title']:
        if isinstance(item['title'], list):
            return item['title'][0]
        return item['title']
    
    # Try language-aware title
    if 'dcTitleLangAware' in item:
        for lang in ['en', 'nl', 'de', 'fr', 'def']:
            if lang in item['dcTitleLangAware']:
                return item['dcTitleLangAware'][lang][0]
    
    return "Untitled"


def get_item_creator(item):
    """Extract creator/artist from an item."""
    if 'dcCreator' in item and item['dcCreator']:
        if isinstance(item['dcCreator'], list):
            return item['dcCreator'][0]
        return item['dcCreator']
    return "Unknown"


def get_item_year(item):
    """Extract year from an item."""
    if 'year' in item and item['year']:
        if isinstance(item['year'], list):
            return item['year'][0]
        return item['year']
    return "n.d."


def get_item_preview(item):
    """Extract preview image URL."""
    if 'edmPreview' in item and item['edmPreview']:
        if isinstance(item['edmPreview'], list):
            return item['edmPreview'][0]
        return item['edmPreview']
    return None


def display_items(items, max_display=5):
    """Display a formatted list of items."""
    for i, item in enumerate(items[:max_display], 1):
        title = get_item_title(item)
        creator = get_item_creator(item)
        year = get_item_year(item)
        country = item.get('country', ['Unknown'])[0] if item.get('country') else 'Unknown'
        provider = item.get('dataProvider', ['Unknown'])[0] if item.get('dataProvider') else 'Unknown'
        
        print(f"{i:2}. {title}")
        print(f"    by {creator} ({year})")
        print(f"    {provider}, {country}")
        print()

---

## Part 6: Example Searches

Let's explore some interesting queries!

In [ ]:
# Example 1: Search for paintings by Rembrandt
print("=" * 60)
print("Example 1: Rembrandt paintings")
print("=" * 60)

results = search_europeana(
    query="Rembrandt",
    rows=5,
    qf=["TYPE:IMAGE"],
    reusability="open"
)

if results and results.get('success'):
    print(f"Found {results['totalResults']:,} items\n")
    display_items(results['items'])
else:
    print("❌ Search failed")

In [ ]:
# Example 2: Dutch artworks from the Rijksmuseum
print("=" * 60)
print("Example 2: Dutch artworks from Rijksmuseum")
print("=" * 60)

results = search_europeana(
    query="painting",
    rows=5,
    qf=["COUNTRY:Netherlands", "DATA_PROVIDER:Rijksmuseum"],
    reusability="open"
)

if results and results.get('success'):
    print(f"Found {results['totalResults']:,} items\n")
    display_items(results['items'])
else:
    print("❌ Search failed")

---

## Exercise 1: Custom Search

**Your task:** Modify the search parameters to find items that interest you!

**Search ideas:**
- Artists: `"Van Gogh"`, `"Monet"`, `"Vermeer"`, `"Leonardo da Vinci"`
- Subjects: `"landscape"`, `"portrait"`, `"still life"`, `"manuscript"`
- Countries: `"Netherlands"`, `"France"`, `"Italy"`, `"Germany"`, `"United Kingdom"`
- Institutions: `"Rijksmuseum"`, `"British Library"`, `"Louvre"`

In [ ]:
# ============================================================
# EXERCISE 1: Modify these search parameters!
# ============================================================

SEARCH_QUERY = "Van Gogh"  # <-- CHANGE THIS!
SEARCH_COUNTRY = "Netherlands"  # <-- CHANGE THIS (or set to None)
SEARCH_ROWS = 10  # <-- How many results?

# ============================================================

# Build query filters
filters = ["TYPE:IMAGE"]
if SEARCH_COUNTRY:
    filters.append(f"COUNTRY:{SEARCH_COUNTRY}")

print(f"Searching for: {SEARCH_QUERY}")
print(f"Filters: {', '.join(filters)}")
print("=" * 60)

my_results = search_europeana(
    query=SEARCH_QUERY,
    rows=SEARCH_ROWS,
    qf=filters,
    reusability="open"
)

if my_results and my_results.get('success'):
    print(f"\nFound {my_results['totalResults']:,} items\n")
    display_items(my_results['items'], max_display=SEARCH_ROWS)
else:
    print("❌ Search failed")

---

## Part 7: Previewing Images

Let's preview some images directly in the notebook.

In [ ]:
def preview_item(item, size=400):
    """
    Display an item's image in the notebook.
    
    Parameters:
        item: A Europeana item dict
        size: Display width in pixels
    """
    title = get_item_title(item)
    creator = get_item_creator(item)
    year = get_item_year(item)
    
    # Get image URL
    image_url = get_item_preview(item)
    
    if not image_url:
        print(f"❌ No preview image available for: {title}")
        return
    
    # Display info
    print(f"{title}")
    print(f"by {creator}, {year}")
    
    # Get rights
    rights = item.get('rights', ['Unknown'])
    if isinstance(rights, list):
        rights = rights[0] if rights else 'Unknown'
    print(f"License: {rights}")
    print()
    
    # Display image
    try:
        display(Image(url=image_url, width=size))
    except Exception as e:
        print(f"❌ Error displaying image: {e}")

In [ ]:
# Preview images from your search
if my_results and my_results.get('items'):
    print("Previewing first 3 items:")
    print("=" * 60)
    
    for item in my_results['items'][:3]:
        preview_item(item)
        print("\n" + "-" * 60 + "\n")
else:
    print("No results to preview. Run Exercise 1 first!")

---

## Part 8: Downloading Images

Now let's download images from your search results.

**How it works:**
- Europeana provides a **reliable thumbnail service** at `api.europeana.eu/thumbnail`
- This service aggregates and caches images from all source institutions
- Images are typically **200-400px** (good for preview and analysis)
- Downloads are **fast and reliable** (unlike accessing source institutions directly)

**For full-resolution images:**
- Check the item's `edmIsShownAt` field for the source institution's website
- Visit the institution directly to download high-resolution versions
- Some institutions may require registration or have usage restrictions

In [ ]:
def sanitize_filename(name):
    """Remove problematic characters from filenames."""
    if not name:
        return "unknown"
    # Keep only alphanumeric, spaces, dots, underscores, hyphens
    safe = "".join(c for c in name if c.isalnum() or c in ' ._-')
    return safe.strip()[:100]  # Limit length


def download_europeana_images(items, output_dir, max_images=10, delay=1.0, size=400):
    """
    Download images from Europeana search results.
    
    Parameters:
        items: List of Europeana items
        output_dir: Directory to save images
        max_images: Maximum number of images to download
        delay: Seconds to wait between downloads
        size: Image size (200, 400) - uses Europeana's thumbnail service
    
    Returns:
        List of downloaded file paths
    """
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    downloaded = []
    errors = []
    
    # Filter items with images
    items_with_images = [item for item in items if get_item_preview(item)]
    
    if not items_with_images:
        print("❌ No items with images found!")
        return []
    
    to_download = items_with_images[:max_images]
    
    print(f"Downloading {len(to_download)} images...")
    print(f"Saving to: {output_dir}")
    print(f"Image size: {size}px (from Europeana thumbnail service)")
    print()
    
    for i, item in enumerate(to_download, 1):
        try:
            # Get metadata
            title = get_item_title(item)
            creator = get_item_creator(item)
            item_id = item.get('id', 'unknown').replace('/', '_')
            
            # Get preview URL
            preview_url = get_item_preview(item)
            
            if not preview_url:
                error_msg = f"No image URL for: {title}"
                print(f"  [{i}/{len(to_download)}] ⚠️  {error_msg}")
                errors.append(error_msg)
                continue
            
            # Use Europeana's thumbnail API with specified size
            # The preview URL is already from api.europeana.eu/thumbnail
            # We can modify it to request a specific size
            if 'api.europeana.eu/thumbnail' in preview_url:
                # Keep the Europeana thumbnail API URL as-is
                # It's reliable and handles all the complexity for us
                image_url = preview_url
                # Note: Europeana thumbnail API provides consistent access
                # Size is typically around 200-400px which is good for previews
            else:
                # Fallback: use the URL as-is
                image_url = preview_url
            
            # Create filename
            safe_creator = sanitize_filename(creator)
            safe_title = sanitize_filename(title)
            filename = f"{safe_creator}_{item_id}_{safe_title[:30]}.jpg"
            filepath = output_dir / filename
            
            # Skip if already exists
            if filepath.exists() and filepath.stat().st_size > 0:
                print(f"  [{i}/{len(to_download)}] ⊙ Already exists: {filename}")
                downloaded.append(filepath)
                continue
            
            # Download from Europeana's thumbnail service
            # Add headers to mimic a browser request
            headers = {
                'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36',
                'Accept': 'image/webp,image/apng,image/*,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.9',
            }
            
            response = requests.get(image_url, headers=headers, stream=True, timeout=30)
            response.raise_for_status()
            
            # Save image
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
            
            # Verify
            if filepath.exists() and filepath.stat().st_size > 0:
                downloaded.append(filepath)
                file_size_kb = filepath.stat().st_size / 1024
                print(f"  [{i}/{len(to_download)}] ✓ {filename} ({file_size_kb:.1f} KB)")
            else:
                error_msg = f"File empty: {filename}"
                print(f"  [{i}/{len(to_download)}] ❌ {error_msg}")
                errors.append(error_msg)
                if filepath.exists():
                    filepath.unlink()  # Delete empty file
        
        except requests.exceptions.Timeout as e:
            error_msg = f"Timeout downloading: {title[:50]}"
            print(f"  [{i}/{len(to_download)}] ⏱️  {error_msg}")
            errors.append(error_msg)
        
        except requests.exceptions.RequestException as e:
            error_msg = f"Network error: {str(e)[:80]}"
            print(f"  [{i}/{len(to_download)}] ❌ {error_msg}")
            errors.append(error_msg)
        
        except Exception as e:
            error_msg = f"Unexpected error: {type(e).__name__}: {str(e)[:80]}"
            print(f"  [{i}/{len(to_download)}] ❌ {error_msg}")
            errors.append(error_msg)
        
        # Be nice to the server
        if i < len(to_download):
            time.sleep(delay)
    
    print()
    print("=" * 60)
    print(f"✓ Downloaded {len(downloaded)}/{len(to_download)} images successfully")
    if errors:
        print(f"⚠️  {len(errors)} items failed")
        print(f"\nNote: Images are downloaded from Europeana's thumbnail service.")
        print(f"For full-resolution images, visit the source institution directly.")
    print(f"Images saved to: {output_dir}")
    print("=" * 60)
    
    return downloaded

---

## Exercise 2: Download Images

**Your task:** Download images from your search results!

In [ ]:
# ============================================================
# EXERCISE 2: Configure your download!
# ============================================================

MAX_IMAGES = 5  # <-- How many images to download?
FOLDER_NAME = f"{SEARCH_QUERY.replace(' ', '_')}_images"  # <-- Folder name

# ============================================================

if my_results and my_results.get('items'):
    download_dir = IMAGES_DIR / FOLDER_NAME
    
    downloaded_files = download_europeana_images(
        my_results['items'],
        download_dir,
        max_images=MAX_IMAGES,
        size=400  # Thumbnail size from Europeana (reliable)
    )
else:
    print("❌ No search results available. Run Exercise 1 first!")
    downloaded_files = []

In [ ]:
# View one of your downloaded images
if downloaded_files:
    from IPython.display import Image as IPImage
    print(f"Viewing first downloaded image: {downloaded_files[0].name}")
    display(IPImage(filename=str(downloaded_files[0]), width=500))
else:
    print("❌ No images were downloaded.")
    print("   Check the error messages above.")

---

## Part 9: Saving Search Results

Save your search results as JSON for later analysis.

In [ ]:
def save_search_results(results, filename):
    """
    Save search results to a JSON file.
    
    Parameters:
        results: Europeana search results dict
        filename: Output filename
    """
    if not results or not results.get('items'):
        print("❌ No results to save")
        return None
    
    output_path = DATA_DIR / filename
    
    with open(output_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    print(f"✓ Saved {len(results['items'])} items to {output_path}")
    return output_path


# Save your search results
if my_results:
    safe_query = sanitize_filename(SEARCH_QUERY)
    save_search_results(my_results, f"{safe_query}_results.json")

---

## Part 10: IIIF Support (Advanced)

**Good news:** Most Europeana items have IIIF manifests! Europeana generates them on-the-fly from the metadata.

### What is IIIF?

**IIIF (International Image Interoperability Framework)** is a set of standards for delivering images over the web:
- **Zoom** into high-resolution images
- **Crop** specific regions
- **Rotate** and manipulate images
- **Compare** images from different institutions
- **Standardized APIs** that work across museums and libraries

### How to Access IIIF Manifests

Europeana's IIIF manifest URLs follow a simple pattern:

```
https://iiif.europeana.eu/presentation/{dataset}/{local_id}/manifest
```

For example, an item with ID `/90402/SK_A_3262` has the manifest:
```
https://iiif.europeana.eu/presentation/90402/SK_A_3262/manifest
```

### What's in a IIIF Manifest?

A manifest contains:
- **Metadata** - title, creator, description
- **Image sequences** - for multi-page items
- **IIIF Image API endpoints** - for accessing high-resolution images
- **Canvas dimensions** - original image sizes
- **Thumbnails** - preview images

### IIIF Image API

The IIIF Image API lets you request images with specific parameters:

```
{base_url}/{region}/{size}/{rotation}/{quality}.{format}
```

Examples:
- `full/full/0/default.jpg` - full resolution
- `full/1000,/0/default.jpg` - width 1000px, maintain aspect ratio
- `square/500,500/0/default.jpg` - 500x500px square crop
- `0,0,1000,1000/500,/90/default.jpg` - crop, resize, rotate 90°

**Note:** Not all items have IIIF Image API support - it depends on whether the source institution provides it. But all items have manifests with basic metadata.

In [ ]:
def get_iiif_manifest_url(item):
    """
    Construct IIIF manifest URL from item ID.
    
    Europeana generates IIIF manifests on-the-fly for all items.
    Pattern: https://iiif.europeana.eu/presentation/{dataset}/{local_id}/manifest
    
    Returns:
        IIIF manifest URL or None if item ID is invalid
    """
    item_id = item.get('id')
    if not item_id:
        return None
    
    # Parse item ID (format: /dataset/local_id)
    parts = item_id.strip('/').split('/')
    if len(parts) >= 2:
        dataset = parts[0]
        local_id = parts[1]
        return f"https://iiif.europeana.eu/presentation/{dataset}/{local_id}/manifest"
    
    return None


def check_iiif_support(item, verify=False):
    """
    Check if an item has IIIF support.
    
    Parameters:
        item: Europeana item dict
        verify: If True, makes HTTP request to verify manifest exists (slower)
    
    Returns:
        Tuple of (has_iiif, manifest_url)
    """
    manifest_url = get_iiif_manifest_url(item)
    
    if not manifest_url:
        return False, None
    
    # Option 1: Assume all items have IIIF (fastest)
    if not verify:
        return True, manifest_url
    
    # Option 2: Verify by checking if manifest exists (slower, requires HTTP request)
    try:
        response = requests.head(manifest_url, timeout=5)
        if response.status_code == 200:
            return True, manifest_url
    except:
        pass
    
    return False, manifest_url


# Check IIIF support in your results
if my_results and my_results.get('items'):
    print("Checking IIIF support in search results:")
    print("=" * 60)
    print()
    
    # Show first few items with IIIF
    shown = 0
    for item in my_results['items'][:5]:
        has_iiif, manifest_url = check_iiif_support(item, verify=False)
        if has_iiif:
            title = get_item_title(item)
            print(f"✓ {title}")
            print(f"  Manifest: {manifest_url}")
            print()
            shown += 1
    
    # Count total
    iiif_items = [(item, get_iiif_manifest_url(item)) 
                  for item in my_results['items'] 
                  if get_iiif_manifest_url(item)]
    
    print("=" * 60)
    print(f"Result: {len(iiif_items)}/{len(my_results['items'])} items have IIIF manifest URLs")
    print()
    print("Note: Europeana generates IIIF manifests on-the-fly for all items.")
    print("      These manifests provide standardized access to images.")
else:
    print("No results to check. Run Exercise 1 first!")

In [ ]:
def get_iiif_image_info(item):
    """
    Fetch IIIF manifest and extract image information.
    
    Returns:
        Dict with image info including high-resolution URLs
    """
    manifest_url = get_iiif_manifest_url(item)
    if not manifest_url:
        return None
    
    try:
        response = requests.get(manifest_url, timeout=10)
        if response.status_code != 200:
            return None
        
        manifest = response.json()
        
        # Extract image info from manifest
        # IIIF Presentation API structure varies, but typically:
        # manifest -> sequences -> canvases -> images
        info = {
            'manifest_url': manifest_url,
            'label': manifest.get('label', 'Untitled'),
            'images': []
        }
        
        # Navigate the manifest structure
        sequences = manifest.get('sequences', [])
        if sequences:
            canvases = sequences[0].get('canvases', [])
            for canvas in canvases:
                images = canvas.get('images', [])
                for img in images:
                    resource = img.get('resource', {})
                    service = resource.get('service', {})
                    
                    # IIIF Image API endpoint
                    if service and '@id' in service:
                        image_base = service['@id']
                        
                        # IIIF Image API allows you to request images at different sizes:
                        # {base}/full/full/0/default.jpg - full resolution
                        # {base}/full/1000,/0/default.jpg - width 1000px
                        # {base}/full/,1000/0/default.jpg - height 1000px
                        
                        info['images'].append({
                            'service': image_base,
                            'full_size': f"{image_base}/full/full/0/default.jpg",
                            'large': f"{image_base}/full/1000,/0/default.jpg",
                            'medium': f"{image_base}/full/500,/0/default.jpg",
                            'thumbnail': f"{image_base}/full/200,/0/default.jpg"
                        })
        
        return info if info['images'] else None
    
    except Exception as e:
        print(f"Error fetching manifest: {e}")
        return None


# Example: Get high-resolution image URLs for first item
if my_results and my_results.get('items'):
    print("Example: Accessing IIIF images")
    print("=" * 60)
    
    item = my_results['items'][0]
    title = get_item_title(item)
    
    print(f"Item: {title}")
    print()
    print("Fetching IIIF manifest...")
    
    iiif_info = get_iiif_image_info(item)
    
    if iiif_info:
        print(f"✓ Found {len(iiif_info['images'])} image(s)")
        print()
        
        if iiif_info['images']:
            img = iiif_info['images'][0]
            print("Available resolutions:")
            print(f"  Thumbnail (200px): {img['thumbnail']}")
            print(f"  Medium (500px):    {img['medium']}")
            print(f"  Large (1000px):    {img['large']}")
            print(f"  Full resolution:   {img['full_size']}")
            print()
            print("IIIF Image API Features:")
            print(f"  Base URL: {img['service']}")
            print(f"  Region:   /{'{region}'}/  - full, square, or x,y,w,h")
            print(f"  Size:     /{'{size}'}/    - full, max, w,h, pct:n")
            print(f"  Rotation: /{'{rotation}'}/ - 0-360 degrees")
            print(f"  Quality:  /{'{quality}'}/  - default, color, gray, bitonal")
            print(f"  Format:   .{'{format}'}   - jpg, png, webp, etc.")
    else:
        print("❌ Could not fetch IIIF manifest")
else:
    print("No results available. Run Exercise 1 first!")

---

## Summary

In this notebook, you learned:

1. **What APIs are** and why they're useful for digital humanities research
2. **How to search** the Europeana collection with 50+ million items
3. **How to filter** by creator, country, institution, and license
4. **How to download** images and metadata
5. **About IIIF** and standardized image access

### Next Steps

With Europeana data, you can:
- **Compare collections** across European institutions
- **Analyze** artistic trends by country and period
- **Build** cross-institutional datasets
- **Use computer vision** (like CLIP) to analyze images
- **Create visualizations** of cultural heritage

### Useful Resources

- **Europeana Portal**: https://www.europeana.eu/
- **API Documentation**: https://pro.europeana.eu/page/apis
- **Get API Key**: https://pro.europeana.eu/page/get-api
- **IIIF Information**: https://iiif.io/
- **Contact**: api@europeana.eu

### License & Attribution

Europeana aggregates content under various licenses. Always check the `rights` field:
- **CC0**: Public domain, no restrictions
- **CC BY**: Credit required
- **CC BY-SA**: Credit + share-alike
- And others...

When using Europeana data, acknowledge both Europeana and the source institution (dataProvider).